In [1]:
# Import required libraries
!pip install dash==1.19.0 
!pip install jupyter_dash 
!pip install --upgrade plotly

import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px

  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84011 sha256=e04e68c007228393d2c052473b2da02f3679f8109498e22bf58d158d365a055b
  Stored in directory: c:\users\80807819\appdata\local\pip\cache\wheels\e7\27\2f\a9f9bf858843b5365bcb311a80f0b2379737caf4ec50b2ae3a
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-py3-none-any.whl size=3527011 sha256=43d8171df661fca3e33e90b3326e7d2be7eb41fd45f1a7fbe3107e54fd58832e
  Stored in directory: c:\users\80807819\appdata\local\pip\cache\wheels\e3\e6\0e\759ca580c568aad2436521fa6df67f53a141dc1ab8f040c4c7
  Created wheel for dash-html-components: filename=dash_html_components-1.1.2-py3-none-any.whl size=427831 sha256=583dd9797de75823efdcba08832087e86657b3715fe45fc37c52ee396fb63a63
  Stored in directory: c:\users\80807819\appdata\local\pip\cache\wheels\8a\bc\a8\7ad1d354777cfe77413fd7a092e07d7cc82f88f98997f4c463
  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-py3-none-any.whl size=1014872 sha

In [98]:
# Read the airline data into pandas dataframe
#spacex_df = pd.read_csv("spacex_launch_dash.csv")

spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [99]:
# Create a dash application
#app = dash.Dash(__name__)
app = JupyterDash(__name__)

In [100]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'ALL sites', 'value': 'ALL sites'}
                                    ],
                                    value='ALL sites'
                                ),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=min_payload,
                                    max=max_payload,
                                    step=0.1,
                                    value=[min_payload, max_payload]
                                ),
                                html.Div(id='output-container-range-slider'),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [101]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output("success-pie-chart", "figure"), 
    [Input("site-dropdown", "value")])
def generate_chart(sitedropdown):
    if sitedropdown == "ALL sites":
        selection_df = spacex_df[spacex_df['class']==1]
        fig = px.pie(selection_df, values='class', names='Launch Site')
    else:
        selection_df = spacex_df[spacex_df['Launch Site']==sitedropdown]
        sucesses = sum(selection_df['class']==1)
        failures = sum(selection_df['class']==0)
        fig = px.pie(values=[sucesses,failures], names=['Sucess','Failures'])
    return fig

In [102]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output("success-payload-scatter-chart", "figure"), 
    [Input("site-dropdown", "value"),
     Input("payload-slider", "value")])

def update_bar_chart(sitedropdown, payloadslider):
   
    if sitedropdown != "ALL sites":
        selection_df = spacex_df[spacex_df['Launch Site']==sitedropdown]
    else:
        selection_df = spacex_df
    
    selection_df = selection_df[(selection_df['Payload Mass (kg)'] >= payloadslider[0])]
    selection_df = selection_df[(selection_df['Payload Mass (kg)'] <= payloadslider[1])]
    
    fig = px.scatter(
        selection_df, x='Payload Mass (kg)', y="class", 
        color="class")
    
    return fig

In [103]:
# Run the app
#if __name__ == '__main__':
#    app.run_server()
    
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)

#Later in browser use the url
#http://localhost:8090
